In [1]:
import os
import sys
import argparse

print(os.getcwd())
os.chdir("../../")
os.getcwd()

sys.argv = ["view", "--config", "config/multi_task_cross_stitch.yaml"]

parser = argparse.ArgumentParser()
parser.add_argument("--config", type=str, required=True, help="Path to the config file")
args = parser.parse_args()

print(args.config)

/data01/dl23vitcas/dl_project/src/multi_task_cross_stitch
config/multi_task_cross_stitch.yaml


In [3]:
from config_experiments import config
from torchvision.transforms import transforms
from dataloader import VOC08Attr
import matplotlib.pyplot as plt
from model import (
    ObjectDetectionModel,
    AttributePredictionModel,
    CrossStitchNet,
    AttributePredictionHead,
    ObjectDetectionHead,
)
from utils import set_device
import torch
from bbox_transform import resize_bounding_boxes, apply_nms
import matplotlib.patches as patches
import torchvision
import torch.nn as nn

In [8]:
device = "cpu"

model_obj = ObjectDetectionModel().to(device)
model_attr = AttributePredictionModel().to(device)
model = CrossStitchNet(model_obj.backbone, model_attr.backbone).to(device)

In [12]:
# optimizer
params = []
cross_stitch_params = []

for name, param in model.named_parameters():
    if "alfa_a" in name or "alfa_b" in name:
        cross_stitch_params.append(
            {"params": param, "lr": config["cross_stitch"]["lr_cross_stitch"]}
        )
    if "weight" in name:
        params.append(
            {
                "params": param,
                "lr": config["optimizer"]["lr_global"]
                * config["optimizer"]["lr_weigth_mult"],
                "weight_decay": config["optimizer"]["weight_decay"],
            }
        )
    elif "bias" in name:
        params.append(
            {
                "params": param,
                "lr": config["optimizer"]["lr_global"]
                * config["optimizer"]["lr_bias_mult"],
                "weight_decay": config["optimizer"]["weight_decay"],
            }
        )

optimizer = torch.optim.SGD(
    params,
    momentum=config["optimizer"]["momentum"],
)
for param_group in cross_stitch_params:
    optimizer.add_param_group(
        {
            "params": param_group["params"],
            "lr": config["cross_stitch"]["lr_cross_stitch"],
        }
    )

In [13]:
optimizer

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 0.001
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0.0005

Parameter Group 1
    dampening: 0
    differentiable: False
    foreach: None
    lr: 0.002
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0.0005

Parameter Group 2
    dampening: 0
    differentiable: False
    foreach: None
    lr: 0.001
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0.0005

Parameter Group 3
    dampening: 0
    differentiable: False
    foreach: None
    lr: 0.002
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0.0005

Parameter Group 4
    dampening: 0
    differentiable: False
    foreach: None
    lr: 0.001
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0.0005

Parameter Group 5
    dampening: 0
    differentiable: False
    foreach: None
    lr: 0.002
    maximize: Fa